In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load API key from .env file

In [2]:
KEY = os.environ.get("GEMINI_API_KEY")

In [3]:
from google import genai
from google.genai import types

In [4]:
client = genai.Client(api_key=KEY)  # here you can also pass the api_key directly using os.environ['GEMINI_API_KEY']

default_model = "gemini-2.5-flash"

Load vector database

In [5]:
from langchain_community.vectorstores import FAISS  # "db" to store and retrieve embeddings
from langchain_huggingface import HuggingFaceEmbeddings

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
db = FAISS.load_local("faiss_index_all", embeddings, allow_dangerous_deserialization=True)  # Load the vector database

GUI

In [8]:
import gradio as gr

In [17]:
def response_stream(inputs, history):
    user_text = ""
    user_image = None
        
    if type(inputs) == dict:
        user_text = inputs.get("text", "").lower()
        files = inputs.get("files", [])
        if files:  # only uses the first uploaded image
            user_image = Image.open(files[0])
    else:
        user_text = inputs.lower()
    
    # special greetings from example file
    if "hej" in user_text and not "hejdå":
        yield "Hej jag är din livsmedels expert. Vad kan jag hjälpa dig med?"
        return
    elif "hejdå" in user_text:
        yield "Hejdå! Ha en fortsatt trevlig dag :)"
        return

    history_text = "Fortsätt konversationen.\n\n"
    for user_msg, bot_msg in history:
        history_text += f"Använadre: {user_msg}\nAssistent: {bot_msg}\n"

    # Add context from RAG
    context = db.similarity_search(user_text, k=10)
    history_text += "\n\n" + "Kontext:\n" + "".join([chunk.page_content for chunk in context]) + "\n"
    print(history_text)

    history_text += f"Använadre: {user_text}\nAssistent:"
    
    contents = []
    if user_image is not None:
        contents.append(user_image)

    contents.append(history_text)

    try:
        gemini_stream = client.models.generate_content_stream(
            model=default_model,
            contents=contents,
            config=types.GenerateContentConfig(
                temperature=0.0,
                max_output_tokens=2000,
                system_instruction="Du är en livsmedels expert med fokus på\
                innehållförteckningar. besvara användarens frågor enligt kontexten",
                thinking_config=types.ThinkingConfig(thinking_budget=0),
                safety_settings=[
                    types.SafetySetting(
                        category="HARM_CATEGORY_DANGEROUS_CONTENT",
                        threshold="BLOCK_NONE"
                    )
                ]
            )
        )

        # Yield chunks for live updates
        partial_response = ""
        for chunk in gemini_stream:
            if chunk.text:
                partial_response += chunk.text
                yield partial_response
                

    except Exception as e:
        # Handle streaming failure without crashing the chatbot
        yield "Ursäkta, kan du repetera dig snälla :)"
        return
    
with gr.Blocks(fill_height=False) as demo:
    chatbot = gr.ChatInterface(
        fn=response_stream,
        multimodal=True,
        title="Din livsmedelsexpert",
    )

# From lab
# This part closes the demo server if it is already running (which
# happens easily in notebooks) and prevents you from opening multiple
# servers at the same time.
if "demo" in locals() and demo.is_running:
    demo.close()
#----------------------------------------------------------------------------------
demo.launch()

/Users/linus/Desktop/LLM/project/LLM-project/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


In [18]:
if "demo" in locals() and demo.is_running:
    demo.close()

Closing server running on port: 7868
